In [1]:
# Intialization
import os
import sys

os.environ["SPARK_HOME"] = "/home/talentum/spark"
os.environ["PYLIB"] = os.environ["SPARK_HOME"] + "/python/lib"
# In below two lines, use /usr/bin/python2.7 if you want to use Python 2
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3.6" 
os.environ["PYSPARK_DRIVER_PYTHON"] = "/usr/bin/python3"
sys.path.insert(0, os.environ["PYLIB"] +"/py4j-0.10.7-src.zip")
sys.path.insert(0, os.environ["PYLIB"] +"/pyspark.zip")

# NOTE: Whichever package you want mention here.
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0 pyspark-shell' 
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.3 pyspark-shell'
# os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages com.databricks:spark-xml_2.11:0.6.0,org.apache.spark:spark-avro_2.11:2.4.0 pyspark-shell'

In [2]:
#Entrypoint 2.x
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().getOrCreate()

# On yarn:
# spark = SparkSession.builder.appName("Spark SQL basic example").enableHiveSupport().master("yarn").getOrCreate()
# specify .master("yarn")

sc = spark.sparkContext

In [3]:
# ✅ Read output of File 01
from pyspark.sql.functions import from_unixtime, col
events_df = spark.read.parquet(
    "file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/processed/events_base"
)

# Convert timestamp to readable datetime
events_df = events_df.withColumn(
    "event_time",
    from_unixtime(col("timestamp") / 1000)
)

# Split events by type
views_df = events_df.filter(col("event_type") == "view")
cart_df  = events_df.filter(col("event_type") == "addtocart")
txn_df   = events_df.filter(col("event_type") == "transaction")

# Persist processed outputs
views_df.write.mode("overwrite").parquet(
    "file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/processed/views"
)

cart_df.write.mode("overwrite").parquet(
    "file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/processed/addtocart"
)

txn_df.write.mode("overwrite").parquet(
    "file:///home/talentum/test-jupyter/ecommerce-clickstream-etl/data/processed/transactions"
)
